<a href="https://colab.research.google.com/github/luck058/yfinance-prediction/blob/main/import_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd

In [ ]:
def help():
    print('# Set up data fetcher using:')
    print('data_fetcher = YahooFinanceDataFetcher(self, ticker_symbol, period="max", interval="1d")\n')
    print('# Get historical data using:')
    print('df_data = data_fetcher.get_data(verbose=False)\n')
    print('# Get financials using:')
    print('df_financials = data_fetcher.get_financials(verbose=False)\n')
    print('# Get actions using:')
    print('df_actions = data_fetcher.get_actions(verbose=False)\n')

In [ ]:
class YahooFinanceDataFetcher:
    """All functions defined in this notebook are added to this class"""
    def __init__(self, ticker_symbol, period="max", interval="1d"):
        self.ticker = yf.Ticker(ticker_symbol)
        self.period = period
        self.interval = interval

In [ ]:
def get_data(self, verbose=False):
    data = self.ticker.history(period=self.period, interval=self.interval)
    df_data = pd.DataFrame(data)

    if verbose:
        display("Historical Data:")
        display(df_data)

    return df_data

In [ ]:
def get_financials(self, verbose=False):
    financials = self.ticker.financials
    df_financials = pd.DataFrame(financials).T

    if verbose:
        print("Financials:")
        display(df_financials)

    return df_financials

In [ ]:
def get_actions(self, verbose=False):
    actions = self.ticker.actions
    df_actions = pd.DataFrame(actions)

    if verbose:
        print("Actions:")
        display(df_actions)

    return df_actions

In [ ]:
YahooFinanceDataFetcher.get_data = get_data
YahooFinanceDataFetcher.get_financials = get_financials
YahooFinanceDataFetcher.get_actions = get_actions

In [ ]:
# data_fetcher = YahooFinanceDataFetcher("AAPL", period="1mo")

# df_data = data_fetcher.get_data()

# display(df_data)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-12-16 00:00:00-05:00,247.990005,251.380005,247.649994,251.039993,51694800,0.0,0.0
2024-12-17 00:00:00-05:00,250.080002,253.830002,249.779999,253.479996,51356400,0.0,0.0
2024-12-18 00:00:00-05:00,252.160004,254.279999,247.740005,248.050003,56774100,0.0,0.0
2024-12-19 00:00:00-05:00,247.500000,252.000000,247.089996,249.789993,60882300,0.0,0.0
2024-12-20 00:00:00-05:00,248.039993,255.000000,245.690002,254.490005,147495300,0.0,0.0
2024-12-23 00:00:00-05:00,254.770004,255.649994,253.449997,255.270004,40858800,0.0,0.0
2024-12-24 00:00:00-05:00,255.490005,258.209991,255.289993,258.200012,23234700,0.0,0.0
2024-12-26 00:00:00-05:00,258.190002,260.100006,257.630005,259.019989,27237100,0.0,0.0
2024-12-27 00:00:00-05:00,257.829987,258.700012,253.059998,255.589996,42355300,0.0,0.0
